In [1]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [2]:
import os
import sys
import json 

In [3]:
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [5]:
sc.uiWebUrl

'http://spark-master-2.newprolab.com:4042'

In [6]:
data = sc.textFile("/labs/laba01/ml-100k/u.data")
item = sc.textFile("/labs/laba01/ml-100k/u.item")

u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.              
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
              
              user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC   
            
u.item     -- Information about the items (movies); this is a tab separated
              list of
              
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.

In [7]:
# film id for calculations
ID = '328'

In [8]:
data.getNumPartitions(), item.getNumPartitions()

(2, 2)

In [9]:
data.take(1)

['196\t242\t3\t881250949']

In [10]:
item.take(1)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0']

In [11]:
item\
    .map(lambda x: x.split('|'))\
    .take(1)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [12]:
item\
    .filter(lambda x: x.split('|')[0]==ID)\
    .take(1)

['328|Conspiracy Theory (1997)|08-Aug-1997||http://us.imdb.com/M/title-exact?Conspiracy+Theory+(1997)|0|1|0|0|0|0|0|0|0|0|0|0|0|1|1|0|1|0|0']

In [13]:
data\
    .filter(lambda x: x.split()[1]==ID)\
    .take(10)

['166\t328\t5\t886397722',
 '119\t328\t4\t876923913',
 '38\t328\t4\t892428688',
 '229\t328\t1\t891632142',
 '111\t328\t4\t891679939',
 '223\t328\t3\t891548959',
 '173\t328\t5\t877557028',
 '181\t328\t3\t878961227',
 '195\t328\t4\t884420059',
 '292\t328\t3\t877560833']

In [14]:
data.map(lambda x: x.split()[2])\
.take(10)

['3', '3', '1', '2', '1', '4', '2', '5', '3', '3']

In [15]:
hist_film = data\
            .filter(lambda x: x.split()[1]==ID)\
            .map(lambda x: (x.split()[2], 1))\
            .sortByKey()\
            .countByKey()
hist_film

defaultdict(int, {'1': 12, '2': 40, '3': 94, '4': 109, '5': 40})

In [16]:
hist_all = data\
            .map(lambda x: (x.split()[2], 1))\
            .sortByKey()\
            .countByKey()
hist_all

defaultdict(int, {'1': 6110, '2': 11370, '3': 27145, '4': 34174, '5': 21201})

In [22]:
dict_ = {
        "hist_film":list(hist_film.values()), 
        "hist_all":list(hist_all.values())
        }

In [23]:
sum(hist_all.values())

100000

In [24]:
with open("/data/home/an.en/lab01.json", "w") as outfile:
    json.dump(dict_, outfile)